In [12]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

np.set_printoptions(linewidth=np.inf)

In [26]:
# Function to build BDT term structure using binomial lattice and annual compounding
def build_BDT_term_structure(rates, volatility, num_steps):
    term_structure = np.zeros((num_steps + 1, num_steps + 1))
    term_structure[0, 0] = rates[0]

    for i in range(1, num_steps + 1):
        for j in range(i + 1):
            term_structure[j, i] = rates[i] * np.exp(volatility * (i-j))
    return term_structure

def elementary_prices_lattice(rates, volatility, Maturity, q = 0.5):
    term_structure = build_BDT_term_structure(rates, volatility, Maturity-1)
    n = Maturity
    elementary = np.zeros((n+1, n+1))
    elementary[0,0] = 1
    for i in range(1, n+1):
        elementary[0,i] = q * elementary[0,i-1] / (1 + term_structure[0,i-1])
        elementary[i,i] = (1-q) * elementary[i-1,i-1] / (1 + term_structure[i-1,i-1])
        for j in range(1, i):
            elementary[j,i] = q * elementary[j,i-1] / (1 + term_structure[j,i-1]) + (1-q) * elementary[j-1,i-1] / (1 + term_structure[j-1,i-1])

    return elementary

# Function to calculate bond price using BDT model considering volatilities and annual compounding
def bond_prices_BDT(rates, volatility, maturity, face_value=100):
    elementary = elementary_prices_lattice(rates, volatility, maturity)
    bond_prices = np.sum(elementary, axis = 0) * face_value
    return bond_prices[1:]

# Objective function to minimize
def objective_function(params, spot_rates, maturities, volatility):
    rates = params
    model_prices = bond_prices_BDT(rates, volatility, max(maturities))
    model_spot_rates = [(100 / price)**(1 / maturity) - 1 for price, maturity in zip(model_prices, maturities)]
    return np.sum((np.array(model_spot_rates)*100 - np.array(spot_rates))**2)


In [27]:

# Market data
spot_rates = [7.3, 7.62, 8.1, 8.45, 9.2, 9.64, 10.12, 10.45, 10.75, 11.22, 11.55, 11.92, 12.2, 12.32]
maturities = list(range(1, 15)) # 1 to 14 years

# Assumed Volatility
volatility = 0.005

# Initial guess for parameteres
initial_guess = np.array([0.05] * 14)

# Minimize the objective function
result = minimize(objective_function, initial_guess, args=(spot_rates, maturities, volatility), method='BFGS')

# Extract the optimized parameteres
optimized_rates = result.x

# Optimal Interest rate term structure


# Display the results
print("Optimized Rates:", optimized_rates)

# updated term structure based on optimized rates
updated_term_structure = build_BDT_term_structure(optimized_rates, volatility, 13)
print("Updated Term Structure:")
print(updated_term_structure)

print("Bond Prices:")
print(bond_prices_BDT(optimized_rates, volatility, 14))

Optimized Rates: [0.07299999 0.07921106 0.0902117  0.09435721 0.12130235 0.1171925  0.12850179 0.12565973 0.1291854  0.15195073 0.14536472 0.15636182 0.15153981 0.13447894]
Updated Term Structure:
[[0.07299999 0.0796081  0.09111835 0.09578323 0.12375282 0.12015924 0.13241525 0.13013569 0.13445756 0.1589447  0.15281773 0.16520261 0.16091051 0.14351041]
 [0.         0.07921106 0.09066389 0.09530551 0.1231356  0.11955995 0.13175483 0.12948664 0.13378695 0.15815196 0.15205554 0.16437866 0.16010796 0.14279465]
 [0.         0.         0.0902117  0.09483017 0.12252146 0.11896364 0.1310977  0.12884082 0.13311968 0.15736317 0.15129716 0.16355882 0.15930942 0.14208246]
 [0.         0.         0.         0.09435721 0.12191038 0.11837031 0.13044385 0.12819822 0.13245575 0.15657832 0.15054257 0.16274307 0.15851486 0.14137382]
 [0.         0.         0.         0.         0.12130235 0.11777993 0.12979326 0.12755883 0.13179512 0.15579738 0.14979173 0.16193138 0.15772427 0.14066871]
 [0.         0.   

In [21]:

# Assumed Volatility
volatility = 0.005

# Initial guess for parameteres
initial_guess = np.array([0.05] * 14)

# build_BDT_term_structure(initial_guess, volatility, 13)

elementray_prices = elementary_prices_lattice(initial_guess, volatility, 14)
print(elementray_prices)

bond_prices_BDT(initial_guess, volatility, 14)

[[1.00000000e+00 4.76190476e-01 2.26703257e-01 1.07902292e-01 5.13450925e-02 2.44265465e-02 1.16176839e-02 5.52421911e-03 2.62612615e-03 1.24811072e-03 5.93038616e-04 2.81711547e-04 1.33788143e-04 6.35215942e-05 3.01519508e-05]
 [0.00000000e+00 4.76190476e-01 4.53460627e-01 3.23784395e-01 2.05454384e-01 1.22191609e-01 6.97482446e-02 3.86976789e-02 2.10269072e-02 1.12439689e-02 5.93692439e-03 3.10263549e-03 1.60763524e-03 8.27006572e-04 4.22808389e-04]
 [0.00000000e+00 0.00000000e+00 2.26757370e-01 3.23861803e-01 3.08292396e-01 2.44500740e-01 1.74475755e-01 1.16177305e-01 7.36566974e-02 4.50196839e-02 2.67455295e-02 1.55321877e-02 8.85395122e-03 4.96938360e-03 2.75267544e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.07979700e-01 2.05602008e-01 2.44618033e-01 2.32774251e-01 1.93769013e-01 1.47438198e-01 1.05147958e-01 7.13995922e-02 4.66534975e-02 2.95529652e-02 1.82479618e-02 1.10283610e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.14189047e-02 1.22367547e

array([95.23809524, 90.69212542, 86.35281897, 82.21127853, 78.25896685, 74.48769325, 70.88960053, 67.45715227, 64.18312069, 61.06057481, 58.08286912, 55.24363261, 52.53675817, 49.95639242])